In [1]:
%matplotlib inline 
import matplotlib.pylab as plt 
import numpy as np 
import pandas as pd 
import seaborn as sns 

In [7]:
full_data = pd.read_csv('./datasets/Droughts_satelite_and_events.csv', index_col=False)
full_data.head()

,Country,District,year,month,day,date,NDVI,EVI,precipitation_per_hour_v1,precipitation_per_hour_v2,...,SPEI_6month,SPEI_7month,SPEI_8month,SPEI_9month,SPEI_10month,SPEI_11month,SPEI_12month,drought_reported,drought_news_article,drought_desinventar
0,Uganda,ABIM,2000,3,1,2000-03-01,0.270712,0.137156,0.031873,5.285528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
1,Uganda,ABIM,2000,4,1,2000-04-01,0.368438,0.184616,0.172345,84.424662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
2,Uganda,ABIM,2000,5,1,2000-05-01,0.624764,0.365380,0.185952,100.613865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
3,Uganda,ABIM,2000,6,1,2000-06-01,0.669868,0.416891,0.205662,91.973897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
4,Uganda,ABIM,2000,7,1,2000-07-01,0.717746,0.458402,0.157562,67.037938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False


In [8]:
Uganda_data = full_data[full_data.Country=='Uganda']

In [9]:
Uganda_data.head()

,Country,District,year,month,day,date,NDVI,EVI,precipitation_per_hour_v1,precipitation_per_hour_v2,...,SPEI_6month,SPEI_7month,SPEI_8month,SPEI_9month,SPEI_10month,SPEI_11month,SPEI_12month,drought_reported,drought_news_article,drought_desinventar
0,Uganda,ABIM,2000,3,1,2000-03-01,0.270712,0.137156,0.031873,5.285528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
1,Uganda,ABIM,2000,4,1,2000-04-01,0.368438,0.184616,0.172345,84.424662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
2,Uganda,ABIM,2000,5,1,2000-05-01,0.624764,0.365380,0.185952,100.613865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
3,Uganda,ABIM,2000,6,1,2000-06-01,0.669868,0.416891,0.205662,91.973897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
4,Uganda,ABIM,2000,7,1,2000-07-01,0.717746,0.458402,0.157562,67.037938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False


In [13]:
Uganda_data.columns

Index(['Country', 'District', 'year', 'month', 'day', 'date', 'NDVI', 'EVI',
       'precipitation_per_hour_v1', 'precipitation_per_hour_v2',
       'surface_temperature_daytime', 'surface_temperature_nighttime',
       'evapotranspiration', 'rainfall', 'SoilMoisture00_10cm',
       'SoilMoisture10_40cm', 'SoilMoisture40_100cm', 'SoilMoisture100_200cm',
       'SoilTemperature00_10cm', 'SoilTemperature10_40cm',
       'SoilTemperature40_100cm', 'SoilTemperature100_200cm',
       'air_temperature', 'wind_speed', 'SPEI_1month', 'SPEI_2month',
       'SPEI_3month', 'SPEI_4month', 'SPEI_5month', 'SPEI_6month',
       'SPEI_7month', 'SPEI_8month', 'SPEI_9month', 'SPEI_10month',
       'SPEI_11month', 'SPEI_12month', 'drought_reported',
       'drought_news_article', 'drought_desinventar'],
      dtype='object')

In [60]:
first_harvest = [6,7]
second_harvest = [11,12]

first_planting = [3,4,5]

second_planting = [8,9,10]

second_id ='_'.join(str(x) for x in second_harvest)

first_id ='_'.join(str(x) for x in first_harvest)


label_list = ['drought_reported', 'drought_news_article', 'drought_desinventar']

feature_list = list(Uganda_data.drop(labels=['Country','District','year','month','day','date',
                                    ]+label_list, axis=1).columns) 

In [76]:
Uganda_seasonal = pd.DataFrame()

count=0

# --- loop over district + year ---- 
for name, group in Uganda_data.groupby(['District','year']):
    
    count += 1
    print(  (count), 'out of ',len(Uganda_data.groupby(['District','year']))     , end='\r'     )

    # --- get data of first harvest season/planting --- 
    temp_first = pd.DataFrame(group[group.month.apply(lambda x: x in first_harvest)][label_list].sum()).T
    temp_features_first = group[group.month.apply(lambda x: x in first_planting)][feature_list]
    temp_first['Season'] = first_id
    for feature in feature_list:
        for i, value in enumerate(temp_features_first[feature]):
            col_name = feature+'_lag'+str(len(first_planting)-i)
            temp_first[col_name]=value
          
        
    # --- get data of second harvest season/planting ---    
    temp_second = pd.DataFrame(group[group.month.apply(lambda x: x in second_harvest)][label_list].sum()).T
    temp_features_second = group[group.month.apply(lambda x: x in second_planting)][feature_list]
    temp_second['Season'] = second_id
    for feature in feature_list:
        for i, value in enumerate(temp_features_second[feature]):
            col_name = feature+'_lag'+str(len(second_planting)-i)
            temp_second[col_name]=value
            
    #combining the two seasons        
    temp = pd.concat([temp_first,temp_second])
    temp['District'] = name[0]
    temp['year'] = name[1]
    
    Uganda_seasonal = pd.concat([Uganda_seasonal, temp])
    
Uganda_seasonal.reset_index(drop=True, inplace=True)

Uganda_seasonal

C:\Users\behro\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\behro\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,District,EVI_lag1,EVI_lag2,EVI_lag3,NDVI_lag1,NDVI_lag2,NDVI_lag3,SPEI_10month_lag1,SPEI_10month_lag2,SPEI_10month_lag3,...,surface_temperature_daytime_lag1,surface_temperature_daytime_lag2,surface_temperature_daytime_lag3,surface_temperature_nighttime_lag1,surface_temperature_nighttime_lag2,surface_temperature_nighttime_lag3,wind_speed_lag1,wind_speed_lag2,wind_speed_lag3,year
0,ABIM,0.365380,0.184616,0.137156,0.624764,0.368438,0.270712,NaN,NaN,NaN,...,304.404134,311.177578,315.658256,292.344175,293.315996,294.617049,3.503192,4.994076,6.794279,2000
1,ABIM,0.356362,0.398800,0.456916,0.594449,0.621502,0.700513,NaN,NaN,NaN,...,302.416509,302.228380,300.157214,290.764633,290.519499,290.596247,3.927143,3.695719,2.923477,2000
2,ABIM,0.441114,0.334390,0.158305,0.705884,0.590362,0.294246,0.072971,0.131476,1.212956,...,301.793161,306.551851,315.265358,291.998724,292.858063,294.444529,3.024279,5.153910,6.539992,2001
3,ABIM,0.327405,0.360157,0.430884,0.554036,0.622010,0.690541,1.184276,0.205166,0.337706,...,303.043352,301.160417,300.042366,290.867492,290.913252,291.171227,3.747200,3.007668,3.130027,2001
4,ABIM,0.414888,0.227817,0.177122,0.661810,0.465133,0.365969,-1.047781,-0.179861,0.782302,...,303.853265,311.598596,310.920768,292.783299,293.772280,293.698790,3.700571,6.761159,5.896890,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,ZOMBO,0.498448,0.496524,0.485545,0.752831,0.698462,0.754131,1.604284,1.495997,1.379838,...,299.825417,299.277495,298.977287,289.943319,289.229627,289.401039,3.027327,2.660146,2.755454,2017
4876,ZOMBO,0.501362,0.469350,0.351262,0.738639,0.737166,0.557395,1.656815,1.635757,1.402834,...,300.544980,300.265703,302.950295,289.578639,290.228895,290.004583,2.880235,2.566893,3.177298,2018
4877,ZOMBO,0.460891,0.441620,0.432471,0.703145,0.698546,0.684429,0.579438,0.440256,0.118501,...,299.702467,300.290378,301.343829,289.374359,289.198420,289.156413,2.566055,2.592321,3.000399,2018
4878,ZOMBO,0.528117,0.387803,0.285175,0.774606,0.656412,0.503454,0.231150,-0.366002,-0.442910,...,301.061603,305.397222,308.976390,290.433524,290.951106,291.229410,2.958724,3.217659,3.904581,2019


In [83]:
new_feature_list = []

for feature in feature_list:
    for i in range(len(first_planting)):
        name = feature+'_lag'+str(i+1)
        new_feature_list.append(name)

new_feature_list = list(np.sort(new_feature_list))

Uganda_seasonal = Uganda_seasonal[['District','year','Season']+new_feature_list+label_list]

In [86]:
Uganda_seasonal.head()


,District,year,Season,EVI_lag1,EVI_lag2,EVI_lag3,NDVI_lag1,NDVI_lag2,NDVI_lag3,SPEI_10month_lag1,...,surface_temperature_daytime_lag3,surface_temperature_nighttime_lag1,surface_temperature_nighttime_lag2,surface_temperature_nighttime_lag3,wind_speed_lag1,wind_speed_lag2,wind_speed_lag3,drought_reported,drought_news_article,drought_desinventar
0,ABIM,2000,6_7,0.365380,0.184616,0.137156,0.624764,0.368438,0.270712,NaN,...,315.658256,292.344175,293.315996,294.617049,3.503192,4.994076,6.794279,0,0,0
1,ABIM,2000,11_12,0.356362,0.398800,0.456916,0.594449,0.621502,0.700513,NaN,...,300.157214,290.764633,290.519499,290.596247,3.927143,3.695719,2.923477,0,0,0
2,ABIM,2001,6_7,0.441114,0.334390,0.158305,0.705884,0.590362,0.294246,0.072971,...,315.265358,291.998724,292.858063,294.444529,3.024279,5.153910,6.539992,0,0,0
3,ABIM,2001,11_12,0.327405,0.360157,0.430884,0.554036,0.622010,0.690541,1.184276,...,300.042366,290.867492,290.913252,291.171227,3.747200,3.007668,3.130027,0,0,0
4,ABIM,2002,6_7,0.414888,0.227817,0.177122,0.661810,0.465133,0.365969,-1.047781,...,310.920768,292.783299,293.772280,293.698790,3.700571,6.761159,5.896890,0,0,0
